In [1]:
from osgeo import gdal, gdalconst
import shutil
import tkinter as tk
from tkinter import filedialog
import os

# The user will download all specified archives and place them in a single directory.

root = tk.Tk()
root.withdraw()

# Prompt user to select directory
directory = filedialog.askdirectory()
extension = ".zip"

# Extract all files in user-selected directory.
for listing in os.listdir(directory):
    if listing.endswith(extension):
        fullpath = os.path.join(directory, listing)
        folder = shutil.unpack_archive(fullpath, directory)
        os.remove(fullpath)
        del folder


In [2]:
# Create a list of the .tif files in the extracted folders
filelist = []
for file in os.listdir(directory):
    full_path = os.path.join(directory, file, file + '.tif')
    filelist.append(full_path)

In [3]:
# Merge all to a single file
vrt_path = os.path.join(directory, 'mrg_vrt.vrt')
vrt = gdal.BuildVRT(vrt_path, filelist, separate=True)

tif = os.path.join(directory, 'merged.tif')
nc = os.path.join(directory, 'merged.nc')
tifset = gdal.Translate(tif, vrt)
ncset = gdal.Translate(nc, tif)

In [4]:
del tifset
del ncset

In [5]:
from netCDF4 import Dataset
import re

#Rename Variables

nameSearch = r"\\US_(.+)\\"

raster = Dataset(nc, mode="a")

for variable, file in enumerate(filelist):
    name = re.search(nameSearch, file).group(1)
    raster.renameVariable("Band" + str(variable + 1), name)
    

In [ ]:
del raster